In [1]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import pickle
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.datasets import cifar100
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
num_classes = 100
batch_size = 128
epochs = 15

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train_onehot = to_categorical(y_train, num_classes)
y_test_onehot = to_categorical(y_test, num_classes)

In [3]:
label_embeddings = pickle.load(open("Data/Embeddings/CIFAR/CIFAR_100_label_to_embedding_google_news.pk", "rb"))
embedding_len = len(label_embeddings[0])

y_train_embeddings = np.zeros((len(y_train), embedding_len))
y_test_embeddings = np.zeros((len(y_test), embedding_len))

for i in range(y_train.shape[0]):
    y_train_embeddings[i] = label_embeddings[int(y_train[i][0])]

for i in range(y_test.shape[0]):
    y_test_embeddings[i] = label_embeddings[int(y_test[i][0])]

In [4]:
# datagen = ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator()
generator_train = datagen.flow(x_train, y_train_embeddings, batch_size=batch_size, seed=0)
generator_test = datagen.flow(x_test, y_test_embeddings, batch_size=batch_size, seed=0)

In [5]:
def _build_model():
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',input_shape=x_train[0].shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.5))

    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dense(embedding_len, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dense(embedding_len))

    return model

In [6]:
def get_model(retrain=True, save=True):
    if retrain:
        model = _build_model()
        model.compile(loss='mse', optimizer='adam', metrics=['cosine_proximity'])
        history = model.fit_generator(generator=generator_train,
                                            steps_per_epoch=x_train.shape[0] // batch_size,
                                            epochs=epochs,
                                            validation_data=generator_test,
                                            validation_steps=x_test.shape[0] // batch_size
                                     )
        if save:
            model.save('models/VGG_CIFAR100_w2v.h5')                                     
    else:
        model = load_model('models/VGG_CIFAR100_w2v.h5')
    return model

model = get_model()
# model = get_model(save=False)

Epoch 1/15
390/390 [==============================] - 26s 66ms/step - loss: 0.0462 - cosine_proximity: -0.3721 - val_loss: 0.0319 - val_cosine_proximity: -0.3918
Epoch 2/15
390/390 [==============================] - 23s 59ms/step - loss: 0.0298 - cosine_proximity: -0.4318 - val_loss: 0.0306 - val_cosine_proximity: -0.4150
Epoch 3/15
390/390 [==============================] - 23s 59ms/step - loss: 0.0291 - cosine_proximity: -0.4511 - val_loss: 0.0302 - val_cosine_proximity: -0.4359
Epoch 4/15
390/390 [==============================] - 23s 60ms/step - loss: 0.0284 - cosine_proximity: -0.4713 - val_loss: 0.0288 - val_cosine_proximity: -0.4665
Epoch 5/15
390/390 [==============================] - 23s 60ms/step - loss: 0.0274 - cosine_proximity: -0.4968 - val_loss: 0.0288 - val_cosine_proximity: -0.4700
Epoch 6/15
390/390 [==============================] - 23s 60ms/step - loss: 0.0262 - cosine_proximity: -0.5264 - val_loss: 0.0266 - val_cosine_proximity: -0.5236
Epoch 7/15
390/390 [========

In [ ]:
history = model.fit_generator(generator=generator_train,
                                    steps_per_epoch=x_train.shape[0] // batch_size,
                                    epochs=epochs,
                                    validation_data=generator_test,
                                    validation_steps=x_test.shape[0] // batch_size
                             )

Epoch 1/15
181/390 [============>.................] - ETA: 12s - loss: 0.0037 - cosine_proximity: -0.9446

In [11]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [12]:
label_embeddings_arr = np.zeros((100, embedding_len))
for i in range(100):
    label_embeddings_arr[i] = label_embeddings[i]

In [21]:
#Top K Predictions
def pred_top_k(pred_test_y, k=5, label_embeddings_arr=label_embeddings_arr):
    sim_table = cosine_similarity(pred_test_y, label_embeddings_arr)
    top_k_guesses = np.argpartition(sim_table,range(99-k+1,100),axis=1)[:,99-k+1:]
    return sim_table, top_k_guesses

pred_test_y = model.predict(x_test)
k = 5
sim_table, top_k_guesses = pred_top_k(pred_test_y, k=k)

#Top prediction
label_predictions = sim_table.argmax(axis=1)

In [22]:
#Accuracy
acc = np.sum((np.squeeze(y_test) == label_predictions)) / float(y_test.shape[0])
print("Accuracy: " + str(acc))

#Top k Accuracy
def acc(top_k=top_k_guesses, y_test=y_test):
    correct = 0
    for i in range(y_test.shape[0]):
        if np.squeeze(y_test)[i] in top_k[i]:
            correct += 1
    return correct/float(y_test.shape[0])

print("Top " + str(k) + " Accuracy: " + str(acc()))

Accuracy: 0.4905
Top 5 Accuracy: 0.6377
